Merge , Update , Standardize audio files and update corresponding  metadata filenames
1. Standardized segmented audio files(saved as standardized_segmented)
2. Verify parameters of random samples and count(426)
3. Save as standardized_merge as sigle audiofile destination
4. Update name of  audio filename updated_metadata from standardized_segmented audio filename as standardized_2
5. Update name of audio filename of filtered label from standardized_audio audio file name as standardized_1
6. Merge standardized_1 and standardized_2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import librosa
import os
import pandas as pd
import librosa
import soundfile as sf
import numpy as np
import shutil
import os

In [ ]:
#standardize audio files with fixed window length into 6sec audio clips, uniformly sampled at 4410HZ, 16bit depth and a stereo channel configuration.
# Define the path to the folder containing audio files
audio_folder = '/content/drive/MyDrive/DL project/Merged and preprocessed audio/Segmenting_longaudio'

# Define output folder for standardized clips
output_folder = '/content/drive/MyDrive/DL project/Merged and preprocessed audio/Standardized_segmented_files'

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Define target parameters
target_sr = 44100  # Sample rate (Hz)
target_duration = 6  # Duration in seconds
target_bit_depth = 16  # Bit depth
target_channels = 2  # Number of channels (stereo)

# Iterate through files in the folder
for filename in os.listdir(audio_folder):
    # Check if the file is an audio file (e.g., WAV, MP3)
    if filename.endswith(('.wav', '.mp3')):  # Add other audio extensions as needed
        # Construct the full file path
        file_path = os.path.join(audio_folder, filename)

        try:
            # Load audio file
            audio_data, sr = librosa.load(file_path, sr=target_sr, mono=False)  # Load with target sample rate, preserve channels

            # Ensure stereo channels (fix)
            if audio_data.ndim == 1:  # Mono audio
              audio_data = np.tile(audio_data, (2, 1))  # Duplicate mono to stereo
            elif audio_data.ndim > 2:  # More than 2 channels
              audio_data = audio_data[:2, :]  # Select only the first two channels

            # # Check and handle audio data shape
            # if audio_data.ndim == 1:  # Mono audio
            #   audio_data = np.expand_dims(audio_data, axis=0)  # Add channel dimension
            # elif audio_data.ndim > 2:  # More than 2 channels
            #   audio_data = audio_data[:2, :]  # Select only the first two channels (stereo)

            # Pad or trim audio to target duration
            target_samples = target_sr * target_duration
            num_samples = audio_data.shape[1]  # Get number of samples in the audio

            if num_samples < target_samples:
                # Pad with silence if shorter
                pad_width = target_samples - num_samples
                audio_data = np.pad(audio_data, pad_width=((0, 0), (0, pad_width)), mode='constant')
            elif num_samples > target_samples:
                # Trim if longer
                audio_data = audio_data[:, :target_samples]

            # Ensure stereo channels
            if audio_data.ndim == 1:
                audio_data = np.tile(audio_data, (2, 1))  # Duplicate mono to stereo

            # Save the standardized clip
            output_filename = os.path.splitext(filename)[0] + '_standardized.wav'
            output_path = os.path.join(output_folder, output_filename)
            sf.write(output_path, audio_data.T, target_sr, subtype='PCM_16', format='WAV')

        except Exception as e:
            print(f"Error processing file {filename}: {e}")

print("Audio standardization complete!")

Audio standardization complete!


In [ ]:
import random

# Define the path to the folder containing standardized audio files
standardized_folder = '/content/drive/MyDrive/DL project/Merged and preprocessed audio/Standardized_segmented_files'

# Define target parameters
target_sr = 44100  # Sample rate (Hz)
target_duration = 6  # Duration in seconds
target_bit_depth = 16  # Bit depth
target_channels = 2  # Number of channels (stereo)

# Get a list of all standardized audio files
audio_files = [f for f in os.listdir(standardized_folder) if f.endswith(('.wav', '.mp3'))]

# Select 30 random audio files
random_samples = random.sample(audio_files, min(30, len(audio_files)))

# Iterate through the random samples and verify features
for filename in random_samples:
    file_path = os.path.join(standardized_folder, filename)

    try:
        # Load audio file
        audio_data, sr = librosa.load(file_path, sr=None, mono=False)  # Load with original sample rate and channels

        # Verify features
        duration = librosa.get_duration(y=audio_data, sr=sr)
        channels = audio_data.shape[0] if audio_data.ndim > 1 else 1
        bit_depth = sf.info(file_path).subtype.split('_')[-1]  # Extract bit depth from subtype

        print(f"File: {filename}")
        print(f"  Duration: {duration:.2f} seconds (Target: {target_duration} seconds)")
        print(f"  Sample Rate: {sr} Hz (Target: {target_sr} Hz)")
        print(f"  Channels: {channels} (Target: {target_channels})")
        print(f"  Bit Depth: {bit_depth} (Target: {target_bit_depth})")
        print("-" * 20)

    except Exception as e:
        print(f"Error verifying file {filename}: {e}")

print("Verification complete!")

File: EP94_N,N,P L L,24,M_2_standardized.wav
  Duration: 6.00 seconds (Target: 6 seconds)
  Sample Rate: 44100 Hz (Target: 44100 Hz)
  Channels: 2 (Target: 2)
  Bit Depth: 16 (Target: 16)
--------------------
File: BP20_Asthma and lung fibrosis,C,A R M,90,M_3_standardized.wav
  Duration: 6.00 seconds (Target: 6 seconds)
  Sample Rate: 44100 Hz (Target: 44100 Hz)
  Channels: 2 (Target: 2)
  Bit Depth: 16 (Target: 16)
--------------------
File: BP106_Asthma,E W,P L U,45,F_1_standardized.wav
  Duration: 6.00 seconds (Target: 6 seconds)
  Sample Rate: 44100 Hz (Target: 44100 Hz)
  Channels: 2 (Target: 2)
  Bit Depth: 16 (Target: 16)
--------------------
File: 131_1b1_Al_sc_Meditron_2_standardized.wav
  Duration: 6.00 seconds (Target: 6 seconds)
  Sample Rate: 44100 Hz (Target: 44100 Hz)
  Channels: 2 (Target: 2)
  Bit Depth: 16 (Target: 16)
--------------------
File: 168_1b1_Al_sc_Meditron_2_standardized.wav
  Duration: 6.00 seconds (Target: 6 seconds)
  Sample Rate: 44100 Hz (Target: 4410

In [ ]:
import os

def count_files(folder_path):
  """Counts the total number of files in a folder.

  Args:
    folder_path: The path to the folder.

  Returns:
    The total number of files in the folder.
  """
  return len(os.listdir(folder_path))

# Example usage:
folder_path = "/content/drive/MyDrive/DL project/Merged and preprocessed audio/Standardized_segmented_files"  # Replace with your folder path
file_count = count_files(folder_path)
print(f"Total number of files: {file_count}")

Total number of files: 426


In [ ]:
import os

def count_files(folder_path):
  """Counts the total number of files in a folder.

  Args:
    folder_path: The path to the folder.

  Returns:
    The total number of files in the folder.
  """
  return len(os.listdir(folder_path))

# Example usage:
folder_path_1 = "/content/drive/MyDrive/DL project/Merged and preprocessed audio/Standardized"  # Replace with your folder path
folder_path_2 = "/content/drive/MyDrive/DL project/Merged and preprocessed audio/Standardized_segmented_files"
file_count_1 = count_files(folder_path_1)
file_count_2 = count_files(folder_path_2)
print(f"Total number of files: {file_count_1}")
print(f"Total number of files: {file_count_2}")

Total number of files: 1234
Total number of files: 426


In [ ]:
import os
import shutil

def merge_audio_files(source_folder1, source_folder2, destination_folder):
  """Merges audio files from two source folders into a destination folder.

  Args:
    source_folder1: The path to the first source folder.
    source_folder2: The path to the second source folder.
    destination_folder: The path to the destination folder.
  """
  for filename in os.listdir(source_folder1):
    if filename.endswith((".wav", ".mp3", ".flac")):  # Adjust audio extensions as needed
      source_path = os.path.join(source_folder1, filename)
      destination_path = os.path.join(destination_folder, filename)
      shutil.copy(source_path, destination_path)  # Copy the file

  for filename in os.listdir(source_folder2):
    if filename.endswith((".wav", ".mp3", ".flac")):  # Adjust audio extensions as needed
      source_path = os.path.join(source_folder2, filename)
      destination_path = os.path.join(destination_folder, filename)
      shutil.copy(source_path, destination_path)  # Copy the file

# Example usage:
source_folder1 = "/content/drive/MyDrive/DL project/Merged and preprocessed audio/Standardized_segmented_files"
source_folder2 = "/content/drive/MyDrive/DL project/Merged and preprocessed audio/Standardized"
destination_folder = "/content/drive/MyDrive/DL project/Merged and preprocessed audio/Standardized_merged"
merge_audio_files(source_folder1, source_folder2, destination_folder)

In [ ]:
folder_path = "/content/drive/MyDrive/DL project/Merged and preprocessed audio/Standardized_merged"
file_count = count_files(folder_path)

print(f"Total number of files: {file_count}")

Total number of files: 1660


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DL project/labels/updated_metadata.csv')  # Load the CSV file into a pandas DataFrame
print(f"Information about CSV file: {'/content/drive/MyDrive/DL project/labels/updated_metadata.csv'}")
print(df.info())  # Display information about the DataFrame (columns, data types, etc.)
print("\nFirst few rows of the DataFrame:")
print(df)

Information about CSV file: /content/drive/MyDrive/DL project/labels/updated_metadata.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 571 entries, 0 to 570
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Audio File Name  571 non-null    object 
 1   Age              571 non-null    float64
 2   Gender           571 non-null    object 
 3   Location         571 non-null    object 
 4   Sound Type       571 non-null    object 
 5   Condition        571 non-null    object 
dtypes: float64(1), object(5)
memory usage: 26.9+ KB
None

First few rows of the DataFrame:
                  Audio File Name   Age Gender Location Sound Type  \
0        101_1b1_Pr_sc_Meditron_1   3.0      F       Al     normal   
1        101_1b1_Pr_sc_Meditron_2   3.0      F       Al     normal   
2        101_1b1_Pr_sc_Meditron_3   3.0      F       Al     normal   
3        116_1b2_Pl_sc_Meditron_1  56.0      M       Tc     normal

In [ ]:
df.value_counts('Condition')

,count
Condition,
asthma,207
pneumonia,153
urti,66
bronchiectasis,48
bronchiolitis,39
lung fibrosis,24
plueral effusion,19
bronchitis,15


In [ ]:
df_1 = pd.read_csv('/content/drive/MyDrive/DL project/labels/filtered_label.csv')  # Load the CSV file into a pandas DataFrame
print(f"Information about CSV file: {'/content/drive/MyDrive/DL project/labels/filtered_label.csv'}")
print(df.info())  # Display information about the DataFrame (columns, data types, etc.)
print("\nFirst few rows of the DataFrame:")
print(df_1)
df_1.value_counts('Condition')

Information about CSV file: /content/drive/MyDrive/DL project/labels/filtered_label.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 571 entries, 0 to 570
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Audio File Name  571 non-null    object 
 1   Age              571 non-null    float64
 2   Gender           571 non-null    object 
 3   Location         571 non-null    object 
 4   Sound Type       571 non-null    object 
 5   Condition        571 non-null    object 
dtypes: float64(1), object(5)
memory usage: 26.9+ KB
None

First few rows of the DataFrame:
     Audio File Name   Age Gender Location Sound Type Condition
0            104_1b1  70.0      F       Lr     normal      copd
1            105_1b1   7.0      F       Tc     normal      urti
2            103_2b2  70.0      F       Ar     wheeze    asthma
3            104_1b1  70.0      F       Ll     normal      copd
4            104_1b1  70.0  

,count
Condition,
copd,820
healthy,140
asthma,97
pneumonia,52
urti,23
bronchiectasis,16
bronchiolitis,13
lung fibrosis,12
bronchitis,9


In [ ]:
import pandas as pd

def merge_csv_files(file_path1, file_path2, output_file_path):
  """Merges two CSV files with the same column names into one.

  Args:
    file_path1: Path to the first CSV file.
    file_path2: Path to the second CSV file.
    output_file_path: Path to save the merged CSV file.
  """
  df1 = pd.read_csv(file_path1)
  df2 = pd.read_csv(file_path2)
  merged_df = pd.concat([df1, df2], ignore_index=True)
  merged_df.to_csv(output_file_path, index=False)

# Example usage:
file_path1 = "/content/drive/MyDrive/DL project/labels/filtered_label.csv"
file_path2 = "/content/drive/MyDrive/DL project/labels/updated_metadata.csv"
output_file_path = "/content/drive/MyDrive/DL project/labels/standardized_merged.csv"
merge_csv_files(file_path1, file_path2, output_file_path)

In [ ]:
s_m_df = pd.read_csv('/content/drive/MyDrive/DL project/labels/standardized_merged.csv')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
  display(s_m_df)

,Audio File Name,Age,Gender,Location,Sound Type,Condition
0,104_1b1,70.00,F,Lr,normal,copd
1,105_1b1,7.00,F,Tc,normal,urti
2,103_2b2,70.00,F,Ar,wheeze,asthma
3,104_1b1,70.00,F,Ll,normal,copd
4,104_1b1,70.00,F,Ar,normal,copd
5,101_1b1,3.00,F,Al,normal,urti
6,104_1b1,70.00,F,Al,normal,copd
7,104_1b1,70.00,F,Pr,normal,copd
8,106_2b1,73.00,F,Pr,crackle,copd
9,107_2b5,75.00,F,Pr,crackle,copd


In [ ]:
s_m_df.value_counts('Condition')

,count
Condition,
copd,820
asthma,304
pneumonia,205
healthy,140
urti,89
bronchiectasis,64
bronchiolitis,52
lung fibrosis,36
plueral effusion,25


In [ ]:
import pandas as pd
import os
import glob

def update_audio_filenames(csv_file_path, audio_folder_path, output_csv_path):
    df = pd.read_csv(csv_file_path)

    for index, row in df.iterrows():
        metadata_filename = row['Audio File Name']
        audio_files = glob.glob(os.path.join(audio_folder_path, metadata_filename + '*.wav'))

        if audio_files:
            df.loc[index, 'Audio File Name'] = os.path.basename(audio_files[0])  # Get filename only




    df.to_csv(output_csv_path, index=False)

# Example usage:
csv_file_path = "/content/drive/MyDrive/DL project/labels/filtered_label.csv"
audio_folder_path = "/content/drive/MyDrive/DL project/Merged and preprocessed audio/Standardized"
output_csv_path = "/content/drive/MyDrive/DL project/labels/standardized1.csv"
update_audio_filenames(csv_file_path, audio_folder_path, output_csv_path)

In [ ]:
s_df = pd.read_csv('/content/drive/MyDrive/DL project/labels/standardized1.csv')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
  display(s_df)

,Audio File Name,Age,Gender,Location,Sound Type,Condition
0,104_1b1_Pl_sc_Litt3200_standardized.wav,70.00,F,Lr,normal,copd
1,105_1b1,7.00,F,Tc,normal,urti
2,103_2b2,70.00,F,Ar,wheeze,asthma
3,104_1b1_Pl_sc_Litt3200_standardized.wav,70.00,F,Ll,normal,copd
4,104_1b1_Pl_sc_Litt3200_standardized.wav,70.00,F,Ar,normal,copd
5,101_1b1_Pr_sc_Meditron_standardized.wav,3.00,F,Al,normal,urti
6,104_1b1_Pl_sc_Litt3200_standardized.wav,70.00,F,Al,normal,copd
7,104_1b1_Pl_sc_Litt3200_standardized.wav,70.00,F,Pr,normal,copd
8,106_2b1_Pl_mc_LittC2SE_standardized.wav,73.00,F,Pr,crackle,copd
9,107_2b5_Ar_mc_AKGC417L_standardized.wav,75.00,F,Pr,crackle,copd


In [ ]:
csv_file_path_1 = "/content/drive/MyDrive/DL project/labels/updated_metadata.csv"
audio_folder_path_1 = "/content/drive/MyDrive/DL project/Merged and preprocessed audio/Standardized_segmented_files"
output_csv_path_1 = "/content/drive/MyDrive/DL project/labels/standardized2.csv"
update_audio_filenames(csv_file_path_1, audio_folder_path_1, output_csv_path_1)

In [ ]:
s2_df = pd.read_csv('/content/drive/MyDrive/DL project/labels/standardized2.csv')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
  display(s2_df)

,Audio File Name,Age,Gender,Location,Sound Type,Condition
0,101_1b1_Pr_sc_Meditron_1_standardized.wav,3.00,F,Al,normal,urti
1,101_1b1_Pr_sc_Meditron_2_standardized.wav,3.00,F,Al,normal,urti
2,101_1b1_Pr_sc_Meditron_3_standardized.wav,3.00,F,Al,normal,urti
3,116_1b2_Pl_sc_Meditron_1_standardized.wav,56.00,M,Tc,normal,bronchiectasis
4,116_1b2_Pl_sc_Meditron_2_standardized.wav,56.00,M,Tc,normal,bronchiectasis
5,116_1b2_Pl_sc_Meditron_3_standardized.wav,56.00,M,Tc,normal,bronchiectasis
6,119_1b1_Ar_sc_Meditron_1_standardized.wav,2.00,F,Ar,normal,urti
7,119_1b1_Ar_sc_Meditron_2_standardized.wav,2.00,F,Ar,normal,urti
8,119_1b1_Ar_sc_Meditron_3_standardized.wav,2.00,F,Ar,normal,urti
9,122_2b1_Al_mc_LittC2SE_1_standardized.wav,66.00,M,Al,wheeze,pneumonia


In [ ]:
file_path1 = "/content/drive/MyDrive/DL project/labels/standardized1.csv"
file_path2 = "/content/drive/MyDrive/DL project/labels/standardized2.csv"
output_file_path = "/content/drive/MyDrive/DL project/labels/standardized_merged.csv"
merge_csv_files(file_path1, file_path2, output_file_path)

In [ ]:
s_m_df = pd.read_csv('/content/drive/MyDrive/DL project/labels/standardized_merged.csv')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
  display(s_m_df)

,Audio File Name,Age,Gender,Location,Sound Type,Condition
0,104_1b1_Pl_sc_Litt3200_standardized.wav,70.00,F,Lr,normal,copd
1,105_1b1,7.00,F,Tc,normal,urti
2,103_2b2,70.00,F,Ar,wheeze,asthma
3,104_1b1_Pl_sc_Litt3200_standardized.wav,70.00,F,Ll,normal,copd
4,104_1b1_Pl_sc_Litt3200_standardized.wav,70.00,F,Ar,normal,copd
5,101_1b1_Pr_sc_Meditron_standardized.wav,3.00,F,Al,normal,urti
6,104_1b1_Pl_sc_Litt3200_standardized.wav,70.00,F,Al,normal,copd
7,104_1b1_Pl_sc_Litt3200_standardized.wav,70.00,F,Pr,normal,copd
8,106_2b1_Pl_mc_LittC2SE_standardized.wav,73.00,F,Pr,crackle,copd
9,107_2b5_Ar_mc_AKGC417L_standardized.wav,75.00,F,Pr,crackle,copd


In [ ]:
s_m_df.value_counts('Condition')

,count
Condition,
copd,820
asthma,304
pneumonia,205
healthy,140
urti,89
bronchiectasis,64
bronchiolitis,52
lung fibrosis,36
plueral effusion,25
